In [1]:
from openai import OpenAI
from typing import List, Dict, Optional, Union

def ask(messages: List[Dict],is_json:bool = False, stop: Optional[Union[str, List]] = None):
        client = OpenAI(api_key='sk-a6bb624707354861abff16885ffe5252', base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

        if is_json:
                response = client.chat.completions.create(
                        model = 'qwen-plus',
                        temperature = 0.7,
                        messages = messages,
                        response_format={ "type": "json_object" },
                        stop = stop
                )
        else:
                response = client.chat.completions.create(
                        model = 'qwen-plus',
                        temperature = 0.7,
                        messages = messages,
                        stop = stop
                )

        return response

In [2]:
## Web Search Tool
from search import internet_search, process_content

class WebSearch:
    def __init__(self, name:str='web_search', threhold:int=8000):
        self.system_prompt = """
You are a Insight Researcher.

1. To find detail informtion for the user query
and summary the content into one sentence as simple as possible
2. If the user's question is about specific numerical values, 
only return the numerical results without any additional explanation.
"""
        self.name = name
        self.description = "the tool use for web search"
        self.threhold = threhold

    def __call__(self, query:str):
        results = internet_search(query)
        all_text = ""
        windows_size = 0
        for item in results:
            if windows_size >= self.threhold:
                break
            page_content = process_content(item['href'])
            for page in page_content:
                if windows_size + len(page) > self.threhold:
                    remaining_space = self.threhold - windows_size
                    all_text += page[:remaining_space].strip() + "\n\n"
                    windows_size = self.threhold
                    break
                else:
                    windows_size += len(page)
                    all_text += page + "\n\n"
            if windows_size >= self.threhold:
                break

        msg = [{"role":"system","content":self.system_prompt},
               {"role":"user", "content": f"The query is {query}, The search results are {all_text}"}]
        
        answer = ask(messages=msg)

        return answer.choices[0].message.content

In [3]:
# 构建system prompt:包含了tools,Question,Thought,Action,Action。Input,Observation 以及 Final Answer几个部分，之后的 LLM 交互就围绕着这几个部分展开

react_prompt = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
"""

In [4]:
#选取的问题例子
query = "2024 UEFA European Championship and 2024 Copa América champion"

In [5]:
search = WebSearch()
available_tools = {search.name: search}

In [6]:
# constructure prompt
user_prompt = react_prompt.format(tools=search.description, 
                                    tool_names=search.name,
                                    input=query,
                                    agent_scratchpad='')
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion



In [7]:
# 第一次交互
messages = [
    {"role":"user", "content":user_prompt},
]

result1 = ask(messages=messages,
              stop=["Observation", " Observation"])#大模型的停止词是"Observation"，小模型的停止词是" Observation"，所以这里用两个停止词都行
intermediate_step1 = result1.choices[0].message.content
print(intermediate_step1)

Thought: To accurately answer this question, I need to find out which countries or teams will be the champions of the 2024 UEFA European Championship and the 2024 Copa América. Since these events are in the future, I will need to look for any predictions, current standings, or relevant information that might help me provide an informed response.
Action: web_search
Action Input: 2024 UEFA European Championship and 2024 Copa América champion predictions



In [8]:
import re
# 正则匹配action和action_input
regex = (
            r"Action\s*\d*\s*:[\s]*(.*?)[\s]*Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        )
action_match = re.search(regex, intermediate_step1, re.DOTALL)
action = action_match.group(1).strip()
action_input = action_match.group(2)
tool_input = action_input.strip(" ").strip('\n').strip('"')
print(action)
print(tool_input)

web_search
2024 UEFA European Championship and 2024 Copa América champion predictions


In [9]:
# 找到对应的 Tool，将 Action Input 输入并执行，从而得到结果
tool_function1 = available_tools[action]
print(tool_input)
tool_result1 = tool_function1(query=str(tool_input))
print(tool_result1)

2024 UEFA European Championship and 2024 Copa América champion predictions
France is predicted to win the 2024 UEFA European Championship, and no clear prediction is available for the 2024 Copa América champion.


In [10]:
#将第一次 Tool 执行之后的结果，跟之前的 user_prompt 进行合并，得到下一次与 LLM 交互的完整提示词
user_prompt += intermediate_step1
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: To accurately answer this question, I need to find out which countries or teams will be the champions of the 2024 UEFA European Championship and the 2024 Copa América. Since these events are in the future, I will need to look for any predictions, current standings, or relevant information that might help me provide an informed r

In [11]:
user_prompt += f"Observation: {tool_result1}\nThought:"
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: To accurately answer this question, I need to find out which countries or teams will be the champions of the 2024 UEFA European Championship and the 2024 Copa América. Since these events are in the future, I will need to look for any predictions, current standings, or relevant information that might help me provide an informed r

In [12]:
# 第2次交互
messages = [
    {"role":"user", "content":user_prompt},
]

result2 = ask(messages=messages,
              stop=["Observation", " Observation"])
intermediate_step2 = result2.choices[0].message.content
print(intermediate_step2)

I found some information on the potential winner of the 2024 UEFA European Championship, but the data for the 2024 Copa América is less clear. I should perform another search to gather more details on the 2024 Copa América.
Action: web_search
Action Input: 2024 Copa América favorite to win



In [13]:
user_prompt += intermediate_step2
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: To accurately answer this question, I need to find out which countries or teams will be the champions of the 2024 UEFA European Championship and the 2024 Copa América. Since these events are in the future, I will need to look for any predictions, current standings, or relevant information that might help me provide an informed r

In [15]:
import re
# parse the 1st result
regex = (
            r"Action\s*\d*\s*:[\s]*(.*?)[\s]*Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        )
action_match = re.search(regex, intermediate_step2, re.DOTALL)
action = action_match.group(1).strip()
action_input = action_match.group(2)
tool_input = action_input.strip(" ").strip('\n').strip('"')
print(action)
print(tool_input)

web_search
2024 Copa América favorite to win


In [16]:
# call tool function
tool_function2 = available_tools[action]
print(tool_function2.name)
print(tool_input)
tool_result2 = search(query=str(tool_input))
print(tool_result2)

web_search
2024 Copa América favorite to win
Argentina is the favorite to win the 2024 Copa América with odds of +175.


In [17]:
user_prompt += f"Observation: {tool_result2}\nThought:"
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: To accurately answer this question, I need to find out which countries or teams will be the champions of the 2024 UEFA European Championship and the 2024 Copa América. Since these events are in the future, I will need to look for any predictions, current standings, or relevant information that might help me provide an informed r

In [18]:
# 第3次交互
messages = [
    {"role":"user", "content":user_prompt},
]

result3 = ask(messages=messages,
              stop=["Observation", " Observation"])
intermediate_step3 = result3.choices[0].message.content
print(intermediate_step3)

I now have predictions for both the 2024 UEFA European Championship and the 2024 Copa América. France is predicted to win the UEFA European Championship, and Argentina is the favorite to win the Copa América.
Final Answer: The predicted champions are France for the 2024 UEFA European Championship and Argentina for the 2024 Copa América.
